In [34]:
import numpy as np
import pandas as pd
import os
import csv, json
import math
import re
from PIL import Image

from collections import Counter

In [92]:
normal_dataset_path = [
    '../dataset/CSV\\invokeCradleCrafter.csv',
    '../dataset/CSV\\InvokeObfuscation.csv',
    '../dataset/CSV\\IseSteroids.csv',
]
attack_dataset_path = [
    '../dataset/attack_decoded.csv',
    '../dataset/attack_original.csv'
]

In [93]:
def rc_block(code):
    return re.sub(r'<#.*?#>', '', code, flags=re.DOTALL)

def rc_line(code):
    return re.sub(r'#.*?', '', code)

In [94]:
def proc_rawcode(filename, i, rawcode:str):
    if '#' in rawcode:
        print(f'#{i} {filename}')
        #print(rawcode)
        code = rc_line(rc_block(rawcode))
        return code
    else:
        return rawcode

In [95]:
for filename in attack_dataset_path:
    dataset = pd.read_csv(filename)
    rows = dataset['bytes'][:]
    
    output_rows = []
    for i, row in enumerate(rows):
        rawcode = bytes(json.loads(row)).decode('utf-8')
        processed = proc_rawcode(filename, i, rawcode)
        processed_code = json.dumps(list(processed.encode('utf-8')))
        output_rows.append({
            'path' : dataset['path'][i],
            'filename' : dataset['filename'][i],
            'bytes': processed_code,
        })
    output_df = pd.DataFrame(output_rows)

    output_dirname, output_filename = os.path.split(filename)
    
    output_path = f'{output_dirname}/rc_{output_filename}'
    output_df.to_csv(output_path, index=False)
    

#14 ../dataset/attack_decoded.csv
#148 ../dataset/attack_decoded.csv
#152 ../dataset/attack_decoded.csv
#160 ../dataset/attack_decoded.csv
#264 ../dataset/attack_decoded.csv
#326 ../dataset/attack_decoded.csv
#469 ../dataset/attack_decoded.csv
#479 ../dataset/attack_decoded.csv
#503 ../dataset/attack_decoded.csv
#551 ../dataset/attack_decoded.csv
#553 ../dataset/attack_decoded.csv
#620 ../dataset/attack_decoded.csv
#755 ../dataset/attack_decoded.csv
#794 ../dataset/attack_decoded.csv
#888 ../dataset/attack_decoded.csv
#901 ../dataset/attack_decoded.csv
#1064 ../dataset/attack_decoded.csv
#1076 ../dataset/attack_decoded.csv
#1093 ../dataset/attack_decoded.csv
#1112 ../dataset/attack_decoded.csv
#1241 ../dataset/attack_decoded.csv
#1242 ../dataset/attack_decoded.csv
#1253 ../dataset/attack_decoded.csv
#1257 ../dataset/attack_decoded.csv
#1294 ../dataset/attack_decoded.csv
#1335 ../dataset/attack_decoded.csv
#1342 ../dataset/attack_decoded.csv
#1359 ../dataset/attack_decoded.csv
#1378 ../